# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [2]:
X = X[X.date > 85]
y = np.where((X.resp > 0), 1, 0)
X.drop(['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id'], axis=1, inplace=True)

In [3]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=200, verbose=0, random_state=42).fit(X, y)

In [4]:
import pickle
with open('clf.pickle', 'wb') as f:
    pickle.dump(clf, f)

## Calculate utility score of training set where `action` = 1 for all `resp` > 0
[training set](https://www.kaggle.com/c/jane-street-market-prediction/data?select=train.csv)

### Utility score
224162.2681796676

## Split training set and fit tree

In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [2]:
Xt = X[X.date >= 460]
yt = np.where((Xt.resp > 0), 1, 0)
X = X[(X.date > 85) & (X.date < 460)]
y = np.where((X.resp > 0), 1, 0)
drop_cols = ['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id']
X.drop(drop_cols, axis=1, inplace=True)
Xt.drop(drop_cols, axis=1, inplace=True)

In [3]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=100, verbose=0, random_state=42).fit(X, y)

In [4]:
clf.score(Xt, yt)

0.5211998142876572

### max_iter=100, verbose=1, random_state=None
0.5455018688476925

### max_iter=100, verbose=0, random_state=42
0.5211998142876572
kaggle 5171.245

### max_iter=100, verbose=1, random_state=1
0.545544076198337

### max_iter=100, verbose=1, random_state=100
0.5440808880426575
kaggle 3745.948

### max_iter=100, verbose=1, random_state=1000
0.545623801193999

In [ ]:
X = pd.concat([example_sample_submission.set_index('ts_id'), example_test.set_index('ts_id')], axis=1)

In [ ]:
X

In [ ]:
X['pj'] = X.weight * X.action
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)

In [ ]:
X['pj'] = X.weight * np.where((X.resp > 0), X.resp, 0)
pi = X.groupby(['date']).pj.sum()
t = pi.sum()/((pi**2).sum()**0.5) * (250/pi.count())**0.5
u = min(max(t, 0), 6) * pi.sum()
print(u)